In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/COVID-19_Radiography_Dataset"

### Important Libraries 

In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
import glob
import matplotlib.pyplot as plt
import cv2

### Image Data Generators 

In [ ]:
train=path+"/Train"
val=path+"/Val"
test=path+"/Test"

In [ ]:
gen_train = ImageDataGenerator(rescale = 1./255., zoom_range = 0.2)
gen_val = ImageDataGenerator(rescale = 1./255.)
gen_test = ImageDataGenerator(rescale = 1./255.)

train_gen = gen_train.flow_from_directory(train, batch_size = 64, class_mode = 'categorical', target_size = (299, 299))
validation_gen = gen_val.flow_from_directory(val, batch_size = 64, class_mode = 'categorical', target_size=(299, 299))
test_gen = gen_test.flow_from_directory(test, shuffle = False, batch_size=64, class_mode = 'categorical', target_size=(299, 299))

Found 5786 images belonging to 2 classes.
Found 722 images belonging to 2 classes.
Found 722 images belonging to 2 classes.


### Transfer Learning with InceptionResNetV2 Base Model



In [ ]:
base_model=tf.keras.applications.InceptionResNetV2(
    weights="imagenet",
    input_shape=(299,299,3),
    classifier_activation="softmax",
)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation= 'relu'))
# model.add(BatchNormalization())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.15)) 
model.add(Dense(2, activation='softmax'))
model.summary()

225209952/225209952 [==============================] - 9s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 1000)             55873736  
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 256)               256256    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 2)                 

In [ ]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=["accuracy"])
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2)
history = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 50, epochs = 12, callbacks = callback)

Epoch 1/12
50/50 [==============================] - 1090s 22s/step - loss: 0.6395 - accuracy: 0.6988 - val_loss: 0.4245 - val_accuracy: 0.8878
Epoch 2/12
50/50 [==============================] - 374s 7s/step - loss: 0.5067 - accuracy: 0.7584 - val_loss: 0.3928 - val_accuracy: 0.8393
Epoch 3/12
50/50 [==============================] - 206s 4s/step - loss: 0.4712 - accuracy: 0.7701 - val_loss: 0.3863 - val_accuracy: 0.8144
Epoch 4/12
50/50 [==============================] - 136s 3s/step - loss: 0.4440 - accuracy: 0.7869 - val_loss: 0.3772 - val_accuracy: 0.8116
Epoch 5/12
50/50 [==============================] - 105s 2s/step - loss: 0.4353 - accuracy: 0.7912 - val_loss: 0.3909 - val_accuracy: 0.8130
Epoch 6/12
50/50 [==============================] - 89s 2s/step - loss: 0.4177 - accuracy: 0.8031 - val_loss: 0.3749 - val_accuracy: 0.7895
Epoch 7/12
50/50 [==============================] - 81s 2s/step - loss: 0.4165 - accuracy: 0.8028 - val_loss: 0.3354 - val_accuracy: 0.8421
Epoch 8/12
50

### Save the Model

In [ ]:
model.save_weights("InceptionResNet.h5")

In [ ]:
y_pred = model.predict(test_gen)

12/12 [==============================] - 302s 27s/step


In [ ]:
y_pred_list = []

for i in range(len(y_pred)):
    y_pred[i] = np.where(y_pred[i] == np.amax(y_pred[i]))
    y_pred_list.append(y_pred[i][0])


In [ ]:
y_actual = []

for i in range(len(y_pred)):
  if i <361 :
    y_actual.append(0)
  else:
    y_actual.append(1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_actual,y_pred_list)

0.7174515235457064